### <b> Model-Agnostic Meta Learning algorithm (MAML) </b>

In [1]:
"""setup information"""
import torch
import matplotlib.pyplot as plt
import maml
import Data
import importlib as imp
import numpy as np
import encoder
"""dataset split as specified by the authors"""
train_class_names = [
    'train', 'skyscraper', 'turtle', 'raccoon', 'spider', 'orange', 'castle', 'keyboard',
    'clock', 'pear', 'girl', 'seal', 'elephant', 'apple', 'aquarium_fish', 'bus',
    'mushroom', 'possum', 'squirrel', 'chair', 'tank', 'plate', 'wolf', 'road', 'mouse',
    'boy', 'shrew', 'couch', 'sunflower', 'tiger', 'caterpillar', 'lion', 'streetcar',
    'lawn_mower', 'tulip', 'forest', 'dolphin', 'cockroach', 'bear', 'porcupine', 'bee',
    'hamster', 'lobster', 'bowl', 'can', 'bottle', 'trout', 'snake', 'bridge',
    'pine_tree', 'skunk', 'lizard', 'cup', 'kangaroo', 'oak_tree', 'dinosaur', 'rabbit',
    'orchid', 'willow_tree', 'ray', 'palm_tree', 'mountain', 'house', 'cloud'
    ]
valid_class_names = [
    'otter', 'motorcycle', 'television', 'lamp', 'crocodile', 'shark', 'butterfly', 'sea',
    'beaver', 'beetle', 'tractor', 'flatfish', 'maple_tree', 'camel', 'crab', 'cattle'
    ]
test_class_names = [
    'baby', 'bed', 'bicycle', 'chimpanzee', 'fox', 'leopard', 'man', 'pickup_truck',
    'plain', 'poppy', 'rocket', 'rose', 'snail', 'sweet_pepper', 'table', 'telephone',
    'wardrobe', 'whale', 'woman', 'worm'
    ]

2023-10-07 18:37:38.744106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# to run: jupyter nbconvert --execute --to notebook TAML_notebook.ipynb
imp.reload(maml)
imp.reload(encoder)
imp.reload(Data)
number_of_inner_gradient_steps: int = 5
ways: int = 5 # 5
max_shots: int = 50 # 50
query: int = 15 # 15
iterations = 7 # epochs， 1000
inner_learning_rate: float = 0.5
outer_learning_rate: float = 1e-3
batch_size = 4 # how many tasks in this batch
total_train_task = batch_size * iterations
valid_interval = 7 # was 50
total_valid_task = batch_size * int(iterations / valid_interval)
total_test_task = 600 # 600
imbalance = False
m = maml.MAML(num_outputs=ways, 
            num_inner_steps=number_of_inner_gradient_steps,
            inner_lr=inner_learning_rate, learn_inner_lrs=False, 
            outer_lr=outer_learning_rate)
# m._view_model(m._meta_parameters)
loss_list, accuracy_list = m.train(
    dataloader_train=Data.get_dataloader(train_class_names, max_shots, query, ways, total_train_task, batch_size, imbalance),
    dataloader_val=Data.get_dataloader(valid_class_names, max_shots, query, ways, total_valid_task, batch_size, imbalance),
    writer=None)
# test_loss, test_accuracy = m.test( 
#     dataloader_test =Data.get_dataloader(test_class_names,  max_shots, query, ways, total_valid_task, batch_size, imbalance))
# plot results:
f, ax = plt.subplots(figsize=(32, 16), nrows=2, ncols=2)
# plot train
ax[0, 0].plot(loss_list)
ax[0, 0].set_title("train loss")
ax[1, 0].plot(accuracy_list)
ax[1, 0].set_title("train accuracy")
# plot valid
# ax[0, 1].plot(valid_loss)
# ax[0, 1].set_title("valid loss")
# ax[1, 1].plot(valid_accuracy)
# ax[1, 1].set_title("valid accuracy")

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 47.54 GiB total capacity; 9.65 GiB already allocated; 8.75 MiB free; 9.84 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF